In [93]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import shutil

sys.path.append('../')
from meta_fusion.benchmarks import *
from meta_fusion.methods import *
from meta_fusion.models import *
from meta_fusion.utils import *
from meta_fusion.third_party import *
from meta_fusion.synthetic_data import PrepareSyntheticData
from meta_fusion.config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
repetition=1
seed=1

# Data model parameters
n = 2000
dim_modalities = [2000, 400]
noise_ratios = [0.3, 0.3]
trans_type = ["linear", "linear", "quadratic"]
dim_latent = [50, 30, 20]
mod_prop = [0, 0, 1, 0]
interactive_prop = 0

#mod_outs = [[0, 60, 80, 100, 120],[0, 60, 80, 100, 120]]
mod_outs = [[0, 50, 60, 80],[0, 60]]
num_modalities = 2
combined_hiddens = [300,200,100]
mod1_hiddens = mod2_hiddens = [[128],[128]]

# data parameters
data_name = 'regression'
exp_name = "negative_learning"
output_dim = 1  # specify the output dimension for regression

extractor_type = 'PCA'
if extractor_type == 'encoder':
    separate=False
    is_mod_static=[False]*num_modalities  
elif extractor_type == "separate":
    separate=True
    is_mod_static=[False]*num_modalities  
elif extractor_type == 'PCA':
    separate=False
    is_mod_static=[True]*num_modalities  
freeze_mod_extractors=[False]*num_modalities

# Load default model configurations 
config = load_config('../experiments_synthetic/config.json')

# Model files directory
config['ckpt_dir'] = f"./checkpoints/{exp_name}/{seed}/"

# Update other training parameters
config['divergence_weight_type'] = 'clustering'
config['rho_list'] = [0,5,10]
config['optimal_k'] = None
config['output_dim'] = output_dim
config["init_lr"] = 0.001
config["epochs"] = 30
#config["epochs"] = 2
config["ensemble_methods"] = [
        ]

In [114]:
data_preparer = PrepareSyntheticData(data_name = data_name, test_size = 0.2, val_size = 0.2)

In [115]:
def run_single_experiment(config, n, random_state):

    config['random_state'] = random_state
    res_list = []
    best_rho = {}
    cohort_pairs = {}
    cluster_idxs = {}


    #----------------#
    # Split dataset  #
    #----------------#
    train_loader, val_loader, test_loader, oracle_train_loader, oracle_val_loader, oracle_test_loader =\
    data_preparer.get_data_loaders(n, trans_type=trans_type, mod_prop=mod_prop, 
                                   interactive_prop = interactive_prop,
                                   dim_modalities=dim_modalities, dim_latent=dim_latent,
                                   noise_ratios=noise_ratios, random_state=random_state)
    # Get data info
    data_info = data_preparer.get_data_info()
    n = data_info[1]
    n_train = data_info[2]
    n_val = data_info[3]
    n_test = data_info[4]

    print(f"Finished splitting {data_name} dataset. Data information are summarized below:\n"
            f"Modality dimensions: {dim_modalities}\n"
            f"Data size: {n}\n"
            f"Train size: {n_train}\n"
            f"Val size: {n_val}\n"
            f"Test size: {n_test}")
    sys.stdout.flush() 


    #----------------------------#
    #  Adversarial Meta Fusion   #
    #----------------------------#
    meta_extractor = Extractors(mod_outs, dim_modalities, train_loader, val_loader)
    if (extractor_type == 'encoder') or (extractor_type == 'separate'):
        _ = meta_extractor.get_encoder_extractors(mod_hiddens, separate=separate, config=extractor_config)
    elif extractor_type == 'PCA':
        _ = meta_extractor.get_PCA_extractors()
    cohort = Cohorts(extractors=meta_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                     is_mod_static=is_mod_static, freeze_mod_extractors=freeze_mod_extractors)

    cohort_models = cohort.get_cohort_models()
    _, dim_pairs = cohort.get_cohort_info()
    trainer = AdversarialTrainer(config, cohort_models, [train_loader, val_loader])

    # Cohort with normal adaptive weights 
    trainer.train_adaptive() 
    res = trainer.test_adaptive(test_loader)
    res_list.append(res)
    
    best_rho['cohort'] = trainer.best_rho
    cohort_pairs['cohort'] = dim_pairs
    cluster_idxs['cohort'] = trainer.cluster_idxs
    print(f"Finished running adaptive meta fusion!")

    # Cohort with no mutual learning
    res = trainer.test_ablation(test_loader)
    res_list.append(res)
    
    cohort_pairs['indep_cohort'] = dim_pairs
    print(f"Finished testing indepedent cohort!")
    
    # Cohort with adversarial weights 
    trainer.train_adversarial() 
    res = trainer.test_adversarial(test_loader)
    res_list.append(res)
    
    best_rho['adversarial_cohort'] = trainer.fixed_rho
    cohort_pairs['adversarial_cohort'] = dim_pairs
    cluster_idxs['adversarial_cohort'] = trainer.cluster_idxs
    print(f"Finished running adversarial meta fusion!")

    results = []
    for res in res_list:
        for method, val in res.items():
            results.append({'Method': method, 'Test_metric': val, 
                            'best_rho':best_rho.get(method), 'cohort_pairs':cohort_pairs.get(method),
                            'cluster_idxs': cluster_idxs.get(method)})
    

    results = pd.DataFrame(results)

    results['random_state']=random_state
    results["dim_modalities"] = [dim_modalities] * len(results)
    results['n'] = n
    results['n_train'] = n_train
    results['n_val'] = n_val
    results['n_test'] = n_test 

    return results

In [116]:
results = []

for i in tqdm(range(1, repetition+1), desc="Repetitions", leave=True, position=0):
    print(f'Running with repetition {i}...')
    random_state = repetition * (seed-1) + i
    set_random_seed(random_state)
    
    # Run experiment
    tmp = run_single_experiment(config, n, random_state)
    
    results.append(tmp)

Repetitions:   0%|                                                                           | 0/1 [00:00<?, ?it/s]

Running with repetition 1...
Finished splitting regression dataset. Data information are summarized below:
Modality dimensions: [2000, 400]
Data size: 2000
Train size: 1280
Val size: 320
Test size: 400
Start training student cohort with normal adaptive weights...
Training with disagreement penalty = 0

Epoch: 1/30 - LR: 0.001000



100%|█████████████████████████████████████████████████████████████████████████| 1280/1280 [00:03<00:00, 373.89it/s]


model_1: train loss: 184.970, train task loss: 184.970 - val loss: 125.662, val task loss: 125.662 [*] Best so far
model_2: train loss: 178.115, train task loss: 178.115 - val loss: 126.519, val task loss: 126.519 [*] Best so far
model_3: train loss: 176.925, train task loss: 176.925 - val loss: 117.450, val task loss: 117.450 [*] Best so far
model_4: train loss: 183.860, train task loss: 183.860 - val loss: 132.346, val task loss: 132.346 [*] Best so far
model_5: train loss: 180.163, train task loss: 180.163 - val loss: 124.063, val task loss: 124.063 [*] Best so far
model_6: train loss: 177.276, train task loss: 177.276 - val loss: 127.754, val task loss: 127.754 [*] Best so far
model_7: train loss: 193.489, train task loss: 193.489 - val loss: 137.464, val task loss: 137.464 [*] Best so far

Epoch: 2/30 - LR: 0.001000



100%|█████████████████████████████████████████████████████████████████████████| 1280/1280 [00:03<00:00, 375.16it/s]


...

model_1: train loss: 27.767, train task loss: 7.263 - val loss: 361.034, val task loss: 56.871
model_2: train loss: 27.316, train task loss: 7.179 - val loss: 308.942, val task loss: 64.426
model_3: train loss: 27.461, train task loss: 7.393 - val loss: 287.428, val task loss: 62.215 [*] Best so far
model_4: train loss: 30.559, train task loss: 7.984 - val loss: 213.245, val task loss: 83.194
model_5: train loss: 26.603, train task loss: 7.289 - val loss: 297.924, val task loss: 65.687
model_6: train loss: 30.982, train task loss: 8.408 - val loss: 222.788, val task loss: 92.738
model_7: train loss: 27.829, train task loss: 7.272 - val loss: 644.398, val task loss: 50.882 [*] Best so far

Epoch: 30/30 - LR: 0.001000



100%|█████████████████████████████████████████████████████████████████████████| 1280/1280 [00:05<00:00, 229.13it/s]


model_1: train loss: 28.731, train task loss: 7.428 - val loss: 349.464, val task loss: 56.030 [*] Best so far
model_2: train loss: 30.762, train task loss: 7.871 - val loss: 314.735, val task loss: 65.094
model_3: train loss: 29.038, train task loss: 7.354 - val loss: 290.420, val task loss: 61.784 [*] Best so far
model_4: train loss: 32.973, train task loss: 8.238 - val loss: 202.768, val task loss: 83.264
model_5: train loss: 28.395, train task loss: 7.463 - val loss: 295.154, val task loss: 64.339 [*] Best so far
model_6: train loss: 33.280, train task loss: 8.545 - val loss: 209.326, val task loss: 89.822
model_7: train loss: 29.463, train task loss: 7.400 - val loss: 639.971, val task loss: 52.279


Repetitions: 100%|██████████████████████████████████████████████████████████████████| 1/1 [10:24<00:00, 624.66s/it]

Method: (best_single), Test_MSE: 54.58601379394531
Method: (cohort), Test_MSE: [57.060401916503906, 72.99630737304688, 71.11065673828125, 93.23255920410156, 71.61067199707031, 110.03755187988281, 54.58601379394531]
Finished running adversarial meta fusion!


In [125]:
results = pd.concat(results, ignore_index=True)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [126]:
results

,Method,Test_metric,best_rho,cohort_pairs,cluster_idxs,random_state,dim_modalities,n,n_train,n_val,n_test
0,best_single,53.324219,NaN,None,None,1,"[2000, 400]",2000,1280,320,400
1,cohort,"[53.32421875, 66.0692138671875, 65.23515319824...",5.0,"[(80, 60), (80, 0), (60, 60), (60, 0), (50, 60...","[0, 6]",1,"[2000, 400]",2000,1280,320,400
2,indep_best_single,54.864677,NaN,None,None,1,"[2000, 400]",2000,1280,320,400
3,indep_cohort,"[54.86467742919922, 73.9986343383789, 68.25230...",NaN,"[(80, 60), (80, 0), (60, 60), (60, 0), (50, 60...",None,1,"[2000, 400]",2000,1280,320,400
4,adversarial_best_single,54.586014,NaN,None,None,1,"[2000, 400]",2000,1280,320,400
5,adversarial_cohort,"[57.060401916503906, 72.99630737304688, 71.110...",10.0,"[(80, 60), (80, 0), (60, 60), (60, 0), (50, 60...","[3, 5]",1,"[2000, 400]",2000,1280,320,400


In [127]:
results["Test_metric"][1]

[53.32421875,
 66.0692138671875,
 65.23515319824219,
 95.41410064697266,
 69.11542510986328,
 121.18408203125,
 50.805694580078125]

In [128]:
results["Test_metric"][3]

[54.86467742919922,
 73.9986343383789,
 68.25230407714844,
 110.51880645751953,
 73.34672546386719,
 123.37451934814453,
 50.90529251098633]

In [129]:
results["Test_metric"][5]

[57.060401916503906,
 72.99630737304688,
 71.11065673828125,
 93.23255920410156,
 71.61067199707031,
 110.03755187988281,
 54.58601379394531]